In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
from statsmodels.datasets import grunfeld
from ipca import InstrumentedPCA
import seaborn as sns
import zipfile as zp
import pickle

Intersection is Defined to Take only charachteristics which:
- Are present from 1970-2020
- Present in such a way that they will not produce singular matrices with other factors for any month
- Highly correlated variables are removed

In [2]:
with open('intersection.pkl', 'rb') as f:
    intersection = pickle.load(f)
intersection.append('DATE')

In [2]:
#Open the zip file in context manager
with zp.ZipFile('Data.zip', 'r') as myzip:
    #List the files in the zip file
    # myzip.printdir()
    # Save printdir to a variable
    files = myzip.namelist()
files_sorted = sorted(files[3:])
files_from_1970 = files_sorted[551:]
# files_from_1970

In [3]:
restimated_data = False
if restimated_data == True:
    Z_list = []
    y_list = []
    file_path = 'Data/'
    for i, file in enumerate(files_from_1970):
        if i != len(files_from_1970) - 1:
            file_full_path = file_path + file
            df = pd.read_csv(file_full_path)
            df.dropna(axis=0, thresh=60, inplace=True)
            df = df[intersection]
            
            df = df.apply(lambda x: (x - x.mean()) / x.std() if (x.name not in ('permno','DATE', 'RET'))  else x)
            df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
            df.fillna(0, inplace=True)
            Z = df.drop(columns=['RET'])
            y = df[['permno','DATE', 'RET']]
            Z_list.append(Z)
            y_list.append(y)
            print(f'{i}th file is done')
            Z = pd.concat(Z_list)

            #Concatonate all the dataframes in the list
            y = pd.concat(y_list)
            # Save Z and y to pickle
            with open('Z_ipca.pkl', 'wb') as f:
                pickle.dump(Z, f)
            with open('y_ipca.pkl', 'wb') as f:
                pickle.dump(y, f)
else:
    with open('Z_ipca.pkl', 'rb') as f:
        Z = pickle.load(f)
    with open('y_ipca.pkl', 'rb') as f:
        y = pickle.load(f)
    y = pd.DataFrame(y)

In [4]:
# Define Z_train and y_train
Z_train = Z[Z['DATE'] < '2010-01-01']
y_train = y[y['DATE'] < '2010-01-01']

# Set Index for Z_train and y_train
Z_train.set_index(['permno', 'DATE'], inplace=True)
y_train.set_index(['permno', 'DATE'], inplace=True)

In [14]:
X_test = Z[Z['DATE'] >= '2010-01-01']
y_test = y[y['DATE'] >= '2010-01-01']

# Save X_test and y_test to pickle
# with open('X_test.pkl', 'wb') as f:
#     pickle.dump(X_test, f)
# with open('y_test.pkl', 'wb') as f:
#     pickle.dump(y_test, f)

In [5]:

def IPCA(X,y,Z=None, factors=3, intercept=False) :
    regr = InstrumentedPCA(n_factors=factors, intercept=intercept)
    regr = regr.fit(X=X, y=y, Gamma=Z)
    Gamma, Factors = regr.get_factors(label_ind=True)
    return Gamma,Factors

### Using 3 Factors

In [8]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'])

The panel dimensions are:
n_samples: 20374 , L: 70 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.8297801612983825
Step 2 - Aggregate Update: 0.7762083979595327
Step 3 - Aggregate Update: 0.01750244797999878
Step 4 - Aggregate Update: 0.015522806456963856
Step 5 - Aggregate Update: 0.015081676039244318
Step 6 - Aggregate Update: 0.013506588536493558
Step 7 - Aggregate Update: 0.012129133863260422
Step 8 - Aggregate Update: 0.011626235872496157
Step 9 - Aggregate Update: 0.011061760657176664
Step 10 - Aggregate Update: 0.010396213498207255
Step 11 - Aggregate Update: 0.009669466741568433
Step 12 - Aggregate Update: 0.008914179021042815
Step 13 - Aggregate Update: 0.00815596175552237
Step 14 - Aggregate Update: 0.007414182346667783
Step 15 - Aggregate Update: 0.006702826831216965
Step 16 - Aggregate Update: 0.006031359311647874
Step 17 - Aggregate Update: 0.0054055630741103936
Step 18 - Aggregate Update: 0.0048283318025579836
Step 19 - Aggregate Update: 0.004300377545076693
Step 20 - Aggregate Update: 0.0038208341432805215
Step 21 - Aggregate Update: 0.00

#### Compare the factors spit out by the library to our calculated ones

In [10]:
X_month = pd.merge(Z[Z['DATE'] == '1970-01-30'], y[y['DATE'] == '1970-01-30'], on=['permno', 'DATE'], how='inner')
X_calc = X_month.drop(columns=['RET','DATE','permno'])
X_calc = X_calc.to_numpy()
ret_month = X_month['RET'].to_numpy()
f = np.linalg.inv(Gamma.values.T @ X_calc.T @ X_calc @ Gamma.values) @ Gamma.values.T @ X_calc.T @ ret_month
f

# Save Gamma to a pickle file
# with open('Gamma.pkl', 'wb') as f:
#     pickle.dump(Gamma, f)

array([ 0.04144602,  0.04146718, -0.01340544])

In [ ]:
Factors

<span style="color:red">This confirms that we use the right equations to get out of sample factors. </span>

In [19]:
# Open X_test and y_test from pickle
with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open('y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

#Open Gamma from pickle
# with open('Gamma.pkl', 'rb') as f:
#     Gamma = pickle.load(f)
X_test = pd.merge(X_test, y_test, on=['permno', 'DATE'], how='inner')
X_test.drop(columns=['index_x', 'index_y'], inplace=True)

In [9]:
def create_total_r2(actual_returns, pred_returns):
    denom = np.sum(actual_returns**2)
    nom = np.sum((actual_returns - pred_returns)**2)
    r_2 = 1 - nom / denom
    return r_2

In [8]:

def f_t_1_calc(Gamma,X_test) :
    Gamma =Gamma.to_numpy()
    # Calculate the factors for each month in the test set
    factors = np.zeros((len(X_test['DATE'].unique()), Gamma.shape[1]))
    # Loop over each out of sample month
    for i,j in enumerate(X_test['DATE'].unique()):
        X_month = X_test[X_test['DATE'] == j]
        X_calc = X_month.drop(columns=['RET','DATE','permno'])
        X_calc = X_calc.to_numpy()
        ret_month = X_month['RET'].to_numpy()
        #Calculate factors from the formula for each month
        factors[i,:] = np.linalg.inv(Gamma.T @ X_calc.T @ X_calc @ Gamma) @ Gamma.T @ X_calc.T @ ret_month
    return factors

def compute_r_squared(Gamma, factors, X_test) :
    Gamma =Gamma.to_numpy()
    returns_test = []
    returns_predicted = []
    for i,j in enumerate(X_test['DATE'].unique()):
        X_month = X_test[X_test['DATE'] == j]
        X_calc = X_month.drop(columns=['RET','DATE','permno'])
        X_calc = X_calc.to_numpy()
        # Use formula to get the predicted returns for each month
        returns_predicted.append(X_calc @ Gamma @ factors[i,:])
        returns_test.append(X_month['RET'].to_numpy())
    returns_predicted = np.concatenate(returns_predicted)
    returns_test = np.concatenate(returns_test)
    r2 = create_total_r2(returns_test, returns_predicted)
    return r2

def calculate_predictive_r_2(full_data, Gamma, start_date='2010-01-01'):
    temp_data= full_data[full_data['DATE'] < start_date]
    in_sample_factors = f_t_1_calc(Gamma,temp_data)
    lambda_array = np.mean(in_sample_factors, axis=0)
    temp_data = full_data[full_data['DATE'] >= start_date]
    # Broadcast lambda_df to the shape of all_data_out_sample_pivoted
    lambda_broadcasted = np.tile(lambda_array,(len(temp_data.DATE.unique()),1))
    return compute_r_squared(Gamma, lambda_broadcasted, temp_data)

In [22]:
factors = f_t_1_calc(Gamma,X_test)
print(f"Out of Sample R^{2} {compute_r_squared(Gamma, factors, X_test)}")

Out of Sample R^2 0.034574267133338554


Ideas to try: 
- Only calculated R^{2} for stocks which are in PCA.
- Use rank instead of z-score,
- Do not fill the Z with 0-s but with median (shouldn't change much)
- Shift the Characteristics or the returns one period forward.

1. a, Only using it for defined stocks

In [23]:
#Open the pca_cols pickle file

with open('pca_columns.pkl', 'rb') as f:
    pca_cols = pickle.load(f)
    
len(pca_cols)

1. b, Estimate the Gamma only based on selected stocks

In [26]:
Gamma_select,Factors_select = IPCA(Z[(Z['DATE'] < '2010-01-01') & (Z['permno'].isin(pca_cols))].set_index(['permno', 'DATE']), y[(y['DATE'] < '2010-01-01') & (y['permno'].isin(pca_cols))].set_index(['permno', 'DATE'])['RET'])

The panel dimensions are:
n_samples: 4164 , L: 70 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.7376681579311966
Step 2 - Aggregate Update: 0.11290958296798911
Step 3 - Aggregate Update: 0.9983653345276058
Step 4 - Aggregate Update: 0.02586921687575142
Step 5 - Aggregate Update: 0.015649299689812896
Step 6 - Aggregate Update: 0.008889475161653886
Step 7 - Aggregate Update: 0.0048759362902837805
Step 8 - Aggregate Update: 0.00262953576826392
Step 9 - Aggregate Update: 0.0014079952071376564
Step 10 - Aggregate Update: 0.000752418025480539
Step 11 - Aggregate Update: 0.0004023494342502362
Step 12 - Aggregate Update: 0.00021643935107518164
Step 13 - Aggregate Update: 0.00011761619516165167
Step 14 - Aggregate Update: 6.40503394573333e-05
Step 15 - Aggregate Update: 3.49598795038597e-05
Step 16 - Aggregate Update: 1.912539986670403e-05
Step 17 - Aggregate Update: 1.0485820297945203e-05
Step 18 - Aggregate Update: 5.760828803103024e-06
-- Convergence Reached --


#### $R^{2}$

In [33]:
factors = f_t_1_calc(Gamma, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma_select,X_test[X_test.permno.isin(pca_cols)])
out_sample_r_2 = compute_r_squared(Gamma_select, factors, X_test[X_test.permno.isin(pca_cols)])

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'), Gamma_select, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.05921412179080643
Out of Sample R^2 0.04195419944011325
Predictive R^2 0.00020204134195411783


<span style="color:red">So clearly this is not the issue. </span>

2. a, Use rank instead of z-score

"We cross-sectionally transform instruments period-byperiod. \
In particular, we calculate stocks’ ranks for each characteristic, then divide ranks by the number of nonmissing observations and subtract 0.5. \
This maps characteristics into the [−0.5,+0.5] interval"

In [ ]:
Z_list_rank = []
y_list_rank = []
file_path = 'Data/'
for i, file in enumerate(files_from_1970):
    if i != len(files_from_1970) - 1:
        file_full_path = file_path + file
        df = pd.read_csv(file_full_path)
        df.dropna(axis=0, thresh=60, inplace=True)
        df = df[intersection]
        # Use a rank based transformation for each column that maps it into the [-0.5, 0.5] interval
        # This means ranking each observation in each column and then dividing by the number of non-missing observations and subtracting 0.5
        df = df.apply(lambda x: (x.rank(ascending=False) / x.count()) - 0.5 if (x.name not in ('permno','DATE', 'RET'))  else x)
        df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
        #Fill missing values with their median
        df.fillna(df.median(), inplace=True)
        Z = df.drop(columns=['RET'])
        y = df[['permno','DATE', 'RET']]
        Z_list_rank.append(Z)
        y_list_rank.append(y)
        print(f'{i}th file is done')

Z_rank = pd.concat(Z_list_rank)

#Concatonate all the dataframes in the list
y_rank = pd.concat(y_list_rank)

In [9]:
Gamma,Factors = IPCA(Z_rank[Z_rank['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y_rank[y_rank['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'])

The panel dimensions are:
n_samples: 20374 , L: 70 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.5740897942517531
Step 2 - Aggregate Update: 1.2164353555491356
Step 3 - Aggregate Update: 0.03460283633160352
Step 4 - Aggregate Update: 0.015763521549142928
Step 5 - Aggregate Update: 0.009071736074828349
Step 6 - Aggregate Update: 0.0062246456820276574
Step 7 - Aggregate Update: 0.0046915199340689245
Step 8 - Aggregate Update: 0.0036974189371545285
Step 9 - Aggregate Update: 0.002978525977646529
Step 10 - Aggregate Update: 0.0024278747986323235
Step 11 - Aggregate Update: 0.0019929582212651065
Step 12 - Aggregate Update: 0.001643417839312017
Step 13 - Aggregate Update: 0.0013594859281994554
Step 14 - Aggregate Update: 0.0011272316570812013
Step 15 - Aggregate Update: 0.0009363233199553844
Step 16 - Aggregate Update: 0.0007788408909952249
Step 17 - Aggregate Update: 0.0006485789518416163
Step 18 - Aggregate Update: 0.000540602549805147
Step 19 - Aggregate Update: 0.00045094554740882087
Step 20 - Aggregate Update: 0.0003770428614774807
Step 21 - Aggregate U

In [11]:
factors = f_t_1_calc(Gamma, pd.merge(Z_rank, y_rank, on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z_rank, y_rank, on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma,pd.merge(Z_rank[Z_rank['DATE'] > '2010-01-01'], y_rank[y_rank['DATE'] > '2010-01-01'], on=['permno', 'DATE'], how='inner'))
out_of_sample_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z_rank[Z_rank['DATE'] > '2010-01-01'], y_rank[y_rank['DATE'] > '2010-01-01'], on=['permno', 'DATE'], how='inner'))

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z_rank, y_rank, on=['permno', 'DATE'], how='inner'), Gamma, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_of_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.05209667094580539
Out of Sample R^2 0.033350624078562596
Predictive R^2 -0.000456640696574917


<span style="color:red">This is also not the issue </span>

3. a, Shifting the returns columns one period ahead.

<span style="color:red">Here the dimensions do not match and will produce error </span>

In [11]:
#Open Z and y from pickle
with open('Z_ipca.pkl', 'rb') as f:
    Z = pickle.load(f)
with open('y_ipca.pkl', 'rb') as f:
    y = pickle.load(f)

# Convert y to a dataframe
y = pd.DataFrame(y)

def shift_unique_column(column):
    unique_values = column.unique()
    value_map = {value: unique_values[(i + 1) % len(unique_values)] for i, value in enumerate(unique_values)}
    return column.map(value_map)

shifted_column = shift_unique_column(y['DATE'])
y['DATE'] = shifted_column

In [14]:
#Drop the rows which has DATE = '1970-01-30'
y = y[y['DATE'] != '1970-01-30']

In [ ]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'])

This Shows that we tried multiple approaches to try to see what might cause IPCA's week performance but without any success

## Use all characteristics

In [3]:
Z_list = []
y_list = []
file_path = 'Data/'
for i, file in enumerate(files_from_1970):
    if i != len(files_from_1970) - 1:
        file_full_path = file_path + file
        df = pd.read_csv(file_full_path)
        df.dropna(axis=0, thresh=60, inplace=True)
        df.drop(columns=['Unnamed: 0','mve0', 'prc', 'SHROUT', 'sic2'], inplace=True)
        df = df.apply(lambda x: (x - x.mean()) / x.std() if (x.name not in ('permno','DATE', 'RET'))  else x)
        df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
        df.fillna(0, inplace=True)
        Z = df.drop(columns=['RET'])
        y = df[['permno','DATE', 'RET']]
        print(f"Shape of Z: {Z.shape}")
        Z_list.append(Z)
        y_list.append(y)
        print(f'{i}th file is done')

Shape of Z: (1664, 96)
0th file is done
Shape of Z: (1669, 96)
1th file is done
Shape of Z: (1674, 96)
2th file is done
Shape of Z: (1688, 96)
3th file is done
Shape of Z: (1695, 96)
4th file is done
Shape of Z: (1698, 96)
5th file is done
Shape of Z: (1813, 96)
6th file is done
Shape of Z: (1810, 96)
7th file is done
Shape of Z: (1810, 96)
8th file is done
Shape of Z: (1815, 96)
9th file is done
Shape of Z: (1821, 96)
10th file is done
Shape of Z: (1814, 96)
11th file is done
Shape of Z: (1814, 96)
12th file is done
Shape of Z: (1816, 96)
13th file is done
Shape of Z: (1819, 96)
14th file is done
Shape of Z: (1831, 96)
15th file is done
Shape of Z: (1829, 96)
16th file is done
Shape of Z: (1826, 96)
17th file is done
Shape of Z: (1914, 96)
18th file is done
Shape of Z: (1918, 96)
19th file is done
Shape of Z: (1927, 96)
20th file is done
Shape of Z: (1934, 96)
21th file is done
Shape of Z: (1942, 96)
22th file is done
Shape of Z: (1942, 96)
23th file is done
Shape of Z: (1950, 96)
24t

In [4]:
#Concatenate all the dataframes in the list
Z = pd.concat(Z_list)

#Concatonate all the dataframes in the list
y = pd.concat(y_list)
y = pd.DataFrame(y)

# Define Z_train and y_train
# Z_train = Z[Z['DATE'] < '2010-01-01']
# y_train = y[y['DATE'] < '2010-01-01']
# Z_train.set_index(['permno', 'DATE'], inplace=True)
# y_train.set_index(['permno', 'DATE'], inplace=True)

### 3 Factors

In [37]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'])

The panel dimensions are:
n_samples: 20374 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.775122406711298
Step 2 - Aggregate Update: 0.7083890265047477
Step 3 - Aggregate Update: 0.020433952648625298
Step 4 - Aggregate Update: 0.015600137129731476
Step 5 - Aggregate Update: 0.015011487977828253
Step 6 - Aggregate Update: 0.0142149700501088
Step 7 - Aggregate Update: 0.01303471590689792
Step 8 - Aggregate Update: 0.011789853147970597
Step 9 - Aggregate Update: 0.01071884449731883
Step 10 - Aggregate Update: 0.01052677134278529
Step 11 - Aggregate Update: 0.010230744718896664
Step 12 - Aggregate Update: 0.009848488040607187
Step 13 - Aggregate Update: 0.009397470308974254
Step 14 - Aggregate Update: 0.008894953703738856
Step 15 - Aggregate Update: 0.008357516319185423
Step 16 - Aggregate Update: 0.007800429335079781
Step 17 - Aggregate Update: 0.007237135152255214
Step 18 - Aggregate Update: 0.006678932115782632
Step 19 - Aggregate Update: 0.006134876679578433
Step 20 - Aggregate Update: 0.0056118638393955825
Step 21 - Aggregate Update: 0.00511482

In [10]:
X_test = Z[Z['DATE'] >= '2010-01-01']
y_test = y[y['DATE'] >= '2010-01-01']
X_test = pd.merge(X_test, y_test, on=['permno', 'DATE'], how='inner')

#####  $R^{2}$

In [39]:
factors = f_t_1_calc(Gamma,pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))
total_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma,X_test)
out_sample_r2 = compute_r_squared(Gamma, factors, X_test)

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z, y, on=['permno', 'DATE'], how='inner'), Gamma, start_date='2010-01-01')

print(f"Total R^{2} {total_r_2}")
print(f"Out of Sample R^{2} {out_sample_r2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.05526857328790058
Out of Sample R^2 0.034783793943788055
Predictive R^2 -0.0011807516772361915


## 5 Factors

In [40]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'], factors=5)

The panel dimensions are:
n_samples: 20374 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.7592553985179097
Step 2 - Aggregate Update: 0.31616908927723486
Step 3 - Aggregate Update: 0.0920048510373026
Step 4 - Aggregate Update: 0.039153588722934285
Step 5 - Aggregate Update: 0.023090655212782002
Step 6 - Aggregate Update: 0.014670961306519098
Step 7 - Aggregate Update: 0.009178474185361774
Step 8 - Aggregate Update: 0.005772586235978372
Step 9 - Aggregate Update: 0.0036820571794344836
Step 10 - Aggregate Update: 0.002390932033138282
Step 11 - Aggregate Update: 0.0015827643908201128
Step 12 - Aggregate Update: 0.0010683177552087142
Step 13 - Aggregate Update: 0.0007347345358156288
Step 14 - Aggregate Update: 0.0005142483661023656
Step 15 - Aggregate Update: 0.00037710708416977684
Step 16 - Aggregate Update: 0.0002803277838060378
Step 17 - Aggregate Update: 0.00020956473327889136
Step 18 - Aggregate Update: 0.00015743631413187714
Step 19 - Aggregate Update: 0.0001187742836037814
Step 20 - Aggregate Update: 8.992773020386619e-05
Step 21 - Aggregate 

##### $R^{2}$

In [41]:
factors = f_t_1_calc(Gamma,pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))
total_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma,X_test)
out_of_sample_r_2 = compute_r_squared(Gamma, factors, X_test)

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z, y, on=['permno', 'DATE'], how='inner'), Gamma, start_date='2010-01-01')

print(f"Total R^{2} {total_r_2}")
print(f"Out of Sample R^{2} {out_of_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.06445116521778638
Out of Sample R^2 0.04332554393340915
Predictive R^2 -0.0018849947078873885


### Model with Intercept

3 Factors

In [6]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'], intercept=True)

The panel dimensions are:
n_samples: 20374 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.7329252396430692
Step 2 - Aggregate Update: 1.1216591320742322
Step 3 - Aggregate Update: 0.3606328140759647
Step 4 - Aggregate Update: 0.12645377566689453
Step 5 - Aggregate Update: 0.05032319989422393
Step 6 - Aggregate Update: 0.028447026619130206
Step 7 - Aggregate Update: 0.017058991679925778
Step 8 - Aggregate Update: 0.00969684479184299
Step 9 - Aggregate Update: 0.005344130702244515
Step 10 - Aggregate Update: 0.0028879563670662356
Step 11 - Aggregate Update: 0.0015398714328427043
Step 12 - Aggregate Update: 0.0008133049719899754
Step 13 - Aggregate Update: 0.00042661981042432795
Step 14 - Aggregate Update: 0.000222663785541255
Step 15 - Aggregate Update: 0.00011578502913603406
Step 16 - Aggregate Update: 6.0043030262441066e-05
Step 17 - Aggregate Update: 3.107272650329507e-05
Step 18 - Aggregate Update: 1.6055367399597786e-05
Step 19 - Aggregate Update: 8.285984199496443e-06
-- Convergence Reached --


In [11]:
factors = f_t_1_calc(Gamma,pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))
total_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma,X_test)
out_of_sample_r_2 = compute_r_squared(Gamma, factors, X_test)

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z, y, on=['permno', 'DATE'], how='inner'), Gamma, start_date='2010-01-01')

print(f"Total R^{2} {total_r_2}")
print(f"Out of Sample R^{2} {out_of_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.061173925246155614
Out of Sample R^2 0.03941071733211621
Predictive R^2 -0.0017760611779209512


5 Factors

In [12]:
Gamma,Factors = IPCA(Z[Z['DATE'] < '2010-01-01'].set_index(['permno', 'DATE']), y[y['DATE'] < '2010-01-01'].set_index(['permno', 'DATE'])['RET'],factors=5, intercept=True)

The panel dimensions are:
n_samples: 20374 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.7387791487293116
Step 2 - Aggregate Update: 1.110029248875054
Step 3 - Aggregate Update: 0.9392118454110874
Step 4 - Aggregate Update: 0.11189015602818364
Step 5 - Aggregate Update: 0.08604323208539486
Step 6 - Aggregate Update: 0.061360947965224705
Step 7 - Aggregate Update: 0.04261959277741337
Step 8 - Aggregate Update: 0.03368848945974351
Step 9 - Aggregate Update: 0.024862307307308762
Step 10 - Aggregate Update: 0.01985591271157358
Step 11 - Aggregate Update: 0.016637630419680915
Step 12 - Aggregate Update: 0.013398061132425143
Step 13 - Aggregate Update: 0.010541124828907386
Step 14 - Aggregate Update: 0.008186035778524947
Step 15 - Aggregate Update: 0.006314707872477121
Step 16 - Aggregate Update: 0.004857028770843763
Step 17 - Aggregate Update: 0.0037330233354593845
Step 18 - Aggregate Update: 0.0028702403742823374
Step 19 - Aggregate Update: 0.0022089033975954547
Step 20 - Aggregate Update: 0.0017018489988008534
Step 21 - Aggregate Update: 0.0013126

In [13]:
factors = f_t_1_calc(Gamma,pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))
total_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z, y, on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma,X_test)
out_of_sample_r_2 = compute_r_squared(Gamma, factors, X_test)

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z, y, on=['permno', 'DATE'], how='inner'), Gamma, start_date='2010-01-01')

print(f"Total R^{2} {total_r_2}")
print(f"Out of Sample R^{2} {out_of_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.06608134771752194
Out of Sample R^2 0.04459080714904984
Predictive R^2 -0.0018976246999786728


### Using only PCA stocks

In [24]:
#Open the pca_cols pickle file

with open('pca_cols_os.pkl', 'rb') as f:
    pca_cols = pickle.load(f)
    
len(pca_cols)

3551

3 Factors

In [25]:
Gamma_select,Factors_select = IPCA(Z[(Z['DATE'] < '2010-01-01') & (Z['permno'].isin(pca_cols))].set_index(['permno', 'DATE']), y[(y['DATE'] < '2010-01-01') & (y['permno'].isin(pca_cols))].set_index(['permno', 'DATE'])['RET'])

The panel dimensions are:
n_samples: 3551 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.6764402865261869
Step 2 - Aggregate Update: 0.8742229111272011
Step 3 - Aggregate Update: 0.04158028350125531
Step 4 - Aggregate Update: 0.7514256630757791
Step 5 - Aggregate Update: 0.024343074774604623
Step 6 - Aggregate Update: 0.014631201944512562
Step 7 - Aggregate Update: 0.008229585919942106
Step 8 - Aggregate Update: 0.004482231506106771
Step 9 - Aggregate Update: 0.0024008396875485616
Step 10 - Aggregate Update: 0.001274269448928722
Step 11 - Aggregate Update: 0.0006728554959041788
Step 12 - Aggregate Update: 0.00035425675619538133
Step 13 - Aggregate Update: 0.00018621611146624195
Step 14 - Aggregate Update: 9.780422515553999e-05
Step 15 - Aggregate Update: 5.135062916988842e-05
Step 16 - Aggregate Update: 2.6959435043938385e-05
Step 17 - Aggregate Update: 1.415583835073786e-05
Step 18 - Aggregate Update: 7.434900554492696e-06
-- Convergence Reached --


#### $R^{2}$

In [26]:
factors = f_t_1_calc(Gamma, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma_select,X_test[X_test.permno.isin(pca_cols)])
out_sample_r_2 = compute_r_squared(Gamma_select, factors, X_test[X_test.permno.isin(pca_cols)])

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'), Gamma_select, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.08034019260223757
Out of Sample R^2 0.05438139060314129
Predictive R^2 0.0005256403558494549


5 Factors

In [27]:
Gamma_select,Factors_select = IPCA(Z[(Z['DATE'] < '2010-01-01') & (Z['permno'].isin(pca_cols))].set_index(['permno', 'DATE']), y[(y['DATE'] < '2010-01-01') & (y['permno'].isin(pca_cols))].set_index(['permno', 'DATE'])['RET'], factors=5)

The panel dimensions are:
n_samples: 3551 , L: 94 , T: 480


[========================================================================] 100%


Step 1 - Aggregate Update: 0.6559223329427746
Step 2 - Aggregate Update: 0.20296720949481725
Step 3 - Aggregate Update: 0.1665856535384893
Step 4 - Aggregate Update: 0.12211244848862482
Step 5 - Aggregate Update: 0.07621552921631362
Step 6 - Aggregate Update: 0.05949375179152014
Step 7 - Aggregate Update: 0.06503487807432803
Step 8 - Aggregate Update: 0.906454231090163
Step 9 - Aggregate Update: 0.06048007698054667
Step 10 - Aggregate Update: 0.05231375153973773
Step 11 - Aggregate Update: 0.044984912196626814
Step 12 - Aggregate Update: 0.7436843192633034
Step 13 - Aggregate Update: 0.03394582712863786
Step 14 - Aggregate Update: 0.02940850534053291
Step 15 - Aggregate Update: 0.02552664181523928
Step 16 - Aggregate Update: 0.024847565563892282
Step 17 - Aggregate Update: 0.02493561444707798
Step 18 - Aggregate Update: 0.024770535433487617
Step 19 - Aggregate Update: 0.025035556051235766
Step 20 - Aggregate Update: 0.027030342314757067
Step 21 - Aggregate Update: 0.034583863928075026


#### $R^{2}$

In [28]:
factors = f_t_1_calc(Gamma, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma_select,X_test[X_test.permno.isin(pca_cols)])
out_sample_r_2 = compute_r_squared(Gamma_select, factors, X_test[X_test.permno.isin(pca_cols)])

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'), Gamma_select, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.08034019260223757
Out of Sample R^2 0.07426707297367285
Predictive R^2 0.0006092154230372682


#### Estimate Gamma based on only the last 10 years and Selected Stocks

3 Factors

In [29]:
Gamma_select,Factors_select = IPCA(Z[(Z['DATE'] >= '2000-01-01') & (Z['DATE'] < '2010-01-01') & (Z['permno'].isin(pca_cols))].set_index(['permno', 'DATE']), y[(y['DATE'] >= '2000-01-01') & (y['DATE'] < '2010-01-01') & (y['permno'].isin(pca_cols))].set_index(['permno', 'DATE'])['RET'], factors=3)

The panel dimensions are:
n_samples: 3551 , L: 94 , T: 120


[========================================================================] 100%


Step 1 - Aggregate Update: 0.6312718920060569
Step 2 - Aggregate Update: 0.587924919753742
Step 3 - Aggregate Update: 0.05643933763902069
Step 4 - Aggregate Update: 0.01891177132753396
Step 5 - Aggregate Update: 0.0073761576864390666
Step 6 - Aggregate Update: 0.003803410500938853
Step 7 - Aggregate Update: 0.0019256831874901459
Step 8 - Aggregate Update: 0.0009758956915180694
Step 9 - Aggregate Update: 0.0004981500963624486
Step 10 - Aggregate Update: 0.0002565608304010425
Step 11 - Aggregate Update: 0.00013330540624445497
Step 12 - Aggregate Update: 6.981979333492427e-05
Step 13 - Aggregate Update: 3.6823456494972095e-05
Step 14 - Aggregate Update: 1.9535157117112067e-05
Step 15 - Aggregate Update: 1.041405361396075e-05
Step 16 - Aggregate Update: 5.5737694810154e-06
-- Convergence Reached --


In [30]:
#### $R^{2}$
factors = f_t_1_calc(Gamma, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma_select,X_test[X_test.permno.isin(pca_cols)])
out_sample_r_2 = compute_r_squared(Gamma_select, factors, X_test[X_test.permno.isin(pca_cols)])

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'), Gamma_select, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.08034019260223757
Out of Sample R^2 0.0566818466448098
Predictive R^2 0.00034636166750401376


5 Factors

In [31]:
Gamma_select,Factors_select = IPCA(Z[(Z['DATE'] >= '2000-01-01') & (Z['DATE'] < '2010-01-01') & (Z['permno'].isin(pca_cols))].set_index(['permno', 'DATE']), y[(y['DATE'] >= '2000-01-01') & (y['DATE'] < '2010-01-01') & (y['permno'].isin(pca_cols))].set_index(['permno', 'DATE'])['RET'], factors=5)

The panel dimensions are:
n_samples: 3551 , L: 94 , T: 120


[========================================================================] 100%


Step 1 - Aggregate Update: 0.8868318275821621
Step 2 - Aggregate Update: 0.23109001578719301
Step 3 - Aggregate Update: 0.11308033483257071
Step 4 - Aggregate Update: 0.06697699054503158
Step 5 - Aggregate Update: 0.03890626515023371
Step 6 - Aggregate Update: 0.030040764641914797
Step 7 - Aggregate Update: 0.023343199684749655
Step 8 - Aggregate Update: 0.018463438148851874
Step 9 - Aggregate Update: 0.01508247038661438
Step 10 - Aggregate Update: 0.01274661987040819
Step 11 - Aggregate Update: 0.011000298537828634
Step 12 - Aggregate Update: 0.009662317301672843
Step 13 - Aggregate Update: 0.00860815507605904
Step 14 - Aggregate Update: 0.007754667927006326
Step 15 - Aggregate Update: 0.007046589578176041
Step 16 - Aggregate Update: 0.006446875896037728
Step 17 - Aggregate Update: 0.005930296202522639
Step 18 - Aggregate Update: 0.0054792993499774845
Step 19 - Aggregate Update: 0.005081370066960633
Step 20 - Aggregate Update: 0.004727337498625195
Step 21 - Aggregate Update: 0.0044102

In [32]:
#### $R^{2}$
factors = f_t_1_calc(Gamma, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))
tota_r_2 = compute_r_squared(Gamma, factors, pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'))

factors = f_t_1_calc(Gamma_select,X_test[X_test.permno.isin(pca_cols)])
out_sample_r_2 = compute_r_squared(Gamma_select, factors, X_test[X_test.permno.isin(pca_cols)])

predictive_r_2 = calculate_predictive_r_2(pd.merge(Z[Z.permno.isin(pca_cols)], y[y.permno.isin(pca_cols)], on=['permno', 'DATE'], how='inner'), Gamma_select, start_date='2010-01-01')

print(f"Total R^{2} {tota_r_2}")
print(f"Out of Sample R^{2} {out_sample_r_2}")
print(f"Predictive R^{2} {predictive_r_2}")

Total R^2 0.08034019260223757
Out of Sample R^2 0.07564042959380335
Predictive R^2 0.0009256171301451621


### Additional Notes

Our Implementation of the ALS algorithm underlying IPCA

In [ ]:
import numpy as np
import pandas as pd

def ALS_panel(X, Z, y, n_factors=3, max_iter=100, tol=1e-5):
    # Initialize Gamma_beta and F
    n_features = Z.shape[1]
    n_obs = len(Z.index.levels[1])  # number of unique dates
    # Initialize Gamma_beta with the first n_factors eigenvectors of the matrix X.T @ X
    Gamma_beta = np.linalg.svd(X.T @ X)[0][:, :n_factors]
    # Gamma_beta = np.random.rand(n_features, n_factors)
    F = np.zeros((n_obs, n_factors))  # one factor vector per date

    dates = Z.index.levels[1]  # assuming that X and y are sorted by date

    for iteration in range(max_iter):
        Gamma_beta_old = Gamma_beta.copy()
        
        # Update factors F for each time period
        for i, date in enumerate(dates):
            if i == len(dates) - 1:
                break
            df = pd.merge(Z.xs(date, level='DATE'), y.xs(dates[i+1], level='DATE'), how='inner', on=['permno'])
            Z_t = df.drop(columns='RET_t').values
            r_t= df['RET_t'].values
            if Z_t.shape[0] >= n_factors:  # Need at least as many observations as factors to solve
                try:
                    # Equation (6)
                    F[i, :] = (np.linalg.inv(Gamma_beta.T @ Z_t.T @ Z_t @ Gamma_beta) @ Gamma_beta.T @ Z_t.T @ r_t)
                    # print(f"F: {F[i, :]}")
                except np.linalg.LinAlgError:
                    F[i, :] = np.zeros(n_factors)  # if singular, set factors to zero

        # Update loadings Gamma_beta
        # print(f"Initial factors: {F}")
        sum_kron = np.zeros((n_features * n_factors, n_features * n_factors))
        sum_vec = np.zeros(n_features * n_factors)
        for i, date in enumerate(dates[:-1]):
            Z_t = Z.xs(date, level='DATE').values
            f_t = F[i, :]
            r_t = y.xs(date, level='DATE').values
            if Z_t.shape[0] >= n_factors:
                # Need at least as many observations as factors to solve
                try:
                    # Equation (7)
                    sum_kron += np.kron(Z_t, f_t.T).T @ np.kron(Z_t, f_t.T)
                    # print(f"sum_kron: {sum_kron}")
                    sum_vec += np.kron(Z_t, f_t.T).T @ r_t
                    
                except ValueError:
                    continue  # if shapes are not aligned, skip this date
        # return Gamma_beta, F
        # Reshape Gamma_beta after solving the vec(Gamma_beta) equation
        try:
            vec_Gamma_beta = np.linalg.inv(sum_kron) @ sum_vec
            # print(f"Vectorized Gamma_beta: {vec_Gamma_beta}")
            Gamma_beta = vec_Gamma_beta.reshape((n_factors, n_features), order='F').T
        except np.linalg.LinAlgError:
            pass  # if singular, skip this iteration

        # Check for convergence
        if np.linalg.norm(Gamma_beta - Gamma_beta_old) < tol:
            print(f"Converged after {iteration} iterations.")
            break

    return Gamma_beta, pd.DataFrame(F, index=dates)




Gamma_beta, F = ALS_panel(X=Z, Z=X_mod, y=y_mod['RET_t'], n_factors=3, max_iter=20, tol=1e-5)
Gamma_beta
factors = f_t_1_calc(Gamma, X_dec,ret_final)
r_2 = compute_r_squared(ret_final, Gamma, factors, X_dec)
r_2